## Entity Matching Example

Using FEBRL synthetic data

### Load Dependencies

In [17]:
import pandas as pd
from pyent.datasets import generate_febrl_data, remove_nan
from pyent.datasets import train_test_validate_stratified_split as ttvs


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Generate Synthetic Data 

In [18]:
master_df = remove_nan(generate_febrl_data(block_col="given_name", init_seed=2))

master_df.head(3)

Before Droping NaN's shape of data is (78962, 23)
After Droping NaN's shape of data is (50116, 23)


,rec_idL,rec_idR,given_name_l,surname_l,street_number_l,address_1_l,address_2_l,suburb_l,postcode_l,state_l,date_of_birth_l,soc_sec_id_l,given_name_r,surname_r,street_number_r,address_1_r,address_2_r,suburb_r,postcode_r,state_r,date_of_birth_r,soc_sec_id_r,labels
2,rec-4609-org,rec-1285-dup-0,sophie,crisci,17,woolner circuit,sefton park,eastern heights,4165,wa,19480428,8614805,sophie,fitzpatrick,5,longerenong street,briars meadows,kincumber,2450,nws,19200110,7018366,no_match
3,rec-33-org,rec-4212-dup-0,ryan,clarke,42,goodchild street,villa 5,sandy bay,2095,nsw,19940903,9100049,ryan,sherrington,17,stopford crescent,lvel 1280,caulfield north,4715,qld,19680507,8315298,no_match
4,rec-3143-org,rec-3143-dup-0,kobe,jolly,26,alroy circuit,salmaldo caravan park,kyabram,6317,sa,19841213,4583639,kobe,jolly,26,alroy circuit,salmaldo caravgan park,kyabram,6317,sa,19841213,4583639,match


### Split Data into Development and Test Sets

In [19]:
X = master_df.loc[:, ~master_df.columns.isin(["labels"])]
y = master_df.loc[:, "labels"]

X_train, X_test, X_val, y_train, y_test, y_val = ttvs(
    features=X, targets=y, test_size=0.1, validate_size=0.2)


### Consolidate Features for Development and Test

Seperate pipelines for textual features, numeric features and catagorical features